In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_community.tools.pubmed.tool import PubmedQueryRun

llm = ChatOpenAI(openai_api_key=os.getenv("API_KEY"))

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    Chat History:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

articles = PubmedQueryRun()

tools = [
    Tool.from_function(
        name="PubMed Search",
        description="Use when needing to find a protein, gene and disease relationship. The question will include the word 'genes', 'disease' and 'proteins. Return a text.",
        func=articles.run,
        return_direct=True,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True,
)

while True:
    q = input("> ")
    response = agent_executor.invoke({"input": q})
    print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: PubMed Search
Action Input: causes of cancerPublished: 2024-04-08
Title: {'i': 'Fn', '#text': 'Engineering lauric acid-based nanodrug delivery systems for restoring chemosensitivity and improving biocompatibility of 5-FU and OxPt against -associated colorectal tumor.'}
Copyright Information: 
Summary::
['Fusobacterium nucleatum', 'Fn', 'Fn', 'Fn', 'Fn', 'in vitro', 'in vivo']
Colorectal cancer (CRC) occurs in the colorectum and ranks second in the global incidence of all cancers, accounting for one of the highest mortalities. Although the combination chemotherapy regimen of 5-fluorouracil (5-FU) and platinum(IV) oxaliplatin prodrug (OxPt) is an effective strategy for CRC treatment in clinical practice, chemotherapy resistance caused by tumor-resided  () could result in treatment failure. To enhance the efficacy and improve the biocompatibility of combination chemotherapy, we developed an antibacterial